In [ ]:
import torch  
import torch.nn.functional as F  
  
def generate_square_subsequent_mask(sz):  
    """  
    生成用于masked self-attention的mask矩阵。  
    sz: 序列长度  
    返回: 一个形状为 (sz, sz) 的tensor，其中上三角为True（或-inf），下三角为False（或0）。  
    """  
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)  
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))  
    return mask  
  
def masked_self_attention(query, key, value, mask):  
    """  
    计算masked self-attention。  
    query, key, value: 形状为 (batch_size, seq_len, depth) 的tensor。  
    mask: 形状为 (seq_len, seq_len) 的tensor，用于在自注意力计算中屏蔽未来信息。  
    返回: attention层的输出。  
    """  
    d_k = query.size(-1)  
    scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))  
    scores.masked_fill_(mask == float('-inf'), float('-1e20'))  # 将mask中的-inf替换为一个非常小的数，以便softmax将其视为0  
  
    attention_weights = F.softmax(scores, dim=-1)  
  
    output = torch.matmul(attention_weights, value)  
    return output  
  
# 示例用法  
batch_size, seq_len, depth = 1, 10, 512  
query = torch.randn(batch_size, seq_len, depth)  
key = torch.randn(batch_size, seq_len, depth)  
value = torch.randn(batch_size, seq_len, depth)  
  
mask = generate_square_subsequent_mask(seq_len)  
  
   
print(output.shape)  # 输出应为 torch.Size([1, 10, 512])